# **Task: Resume Matching with Job Descriptions Using PDF CVs**
## **For AI/ML Engineering Intern role at Capital Placement**
## **By- Ligandro Singh Yumnam**


# 1)PDF Data **Extraction**

In [ ]:
## Get dataset
!mkdir ~/.kaggle


In [ ]:
# configuring the path of Kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d snehaanbhawal/resume-dataset

 98% 61.0M/62.5M [00:04<00:00, 22.2MB/s]
100% 62.5M/62.5M [00:04<00:00, 15.7MB/s]


In [ ]:
# extracting the compessed Dataset
from zipfile import ZipFile
dataset = '/content/resume-dataset.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [ ]:
# Building a pdf extractor for one single pdf

### **METHOD 1 - Using pyreparser**

In [ ]:
!pip install pyresparser

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 16.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 7.8 MB/s eta 0:00:00
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3959 sha256=a5ce148250e9a272f8bc7bd9d7a4d4cad62792b1f406efe351eae562fe511e31
  Stored in directory: /root/.cache/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9a199cb02352966
Successfully built docx2txt


In [ ]:
from pyresparser import ResumeParser
data = ResumeParser('/content/data/data/ARTS/14879257.pdf').get_extracted_data()

/usr/local/lib/python3.10/dist-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_training' (0.0.0) requires spaCy v2.1 and is incompatible with the current spaCy version (2.3.9). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
data

{'name': 'Professional Experience',
 'email': None,
 'mobile_number': None,
 'skills': ['Reports',
  'Math',
  'Mvp',
  'Partnerships',
  'International',
  'Writing',
  'Research',
  'Interactive',
  'Teaching',
  'Communication',
  'Improvement',
  'Spanish',
  'Analyze',
  'Plan',
  'Administration',
  'Analyze data',
  'Presentations',
  'Training',
  'Supervisor',
  'Strategy',
  'English'],
 'college_name': None,
 'degree': None,
 'designation': None,
 'experience': ['internship, Winter Term: Paris 2007, London 2006; Elon Volunteers: Boys and Girls Club Alamance County Community College',
  'Affiliations',
  'Association of Texas Professional Educators (ATPE) LANGUAGES English and Spanish'],
 'company_names': None,
 'no_of_pages': 2,
 'total_experience': 0.0}

### REMARK :
 1. Able to recognize skills,experience and other key features.
 2. Unable to recognize college_name and degree.

### METHOD 2 -

In [ ]:
from PyPDF2 import PdfReader
# Open the PDF file
pdf_file = open('/content/data/data/ACCOUNTANT/10554236.pdf', 'rb')

# Create a PDF reader object
pdf_text = ""
pdf_reader = PdfReader(pdf_file)

for page_num in range(len(pdf_reader.pages)):
    page = pdf_reader.pages[page_num]
    pdf_text += page.extract_text()

# Close the PDF file
pdf_file.close()

# Print the extracted text
#print(pdf_text)

In [ ]:
import spacy
import re


# Load the English language model
nlp = spacy.load("en_core_web_sm")

# Input text
text = """
ACCOUNTANT Summary
Financial Accountant specializing in financial planning, reporting and analysis within the Department of Defense.
Highlights
Account reconciliations
Results-oriented Financial reporting Critical thinking
Accomplishments
Accounting operations professional
Analysis of financial systems
ERP (Enterprise Resource Planning) software. Excellent facilitator
Served on a tiger team which identified and resolved General Ledger postings in DEAMS totaling $360B in accounting adjustments. This allowed for the first successful fiscal year-end close for 2012.
... (rest of the text) ...
Education
Northern Maine Community College 1994 Associate : Accounting City , State , USA
Emphasis in Business
1994 Associates : Accounting City , State , USA GPA: GPA: 3.41 Accounting GPA: 3.41 174 Hours, Quarter
Attended Husson College, major Accounting 78 semester hours toward Bachelors degree.
"""

# Process the text using spaCy
doc = nlp(text)

# Initialize variables to store extracted information
category = ""
skills = []
education = {}

# Flag to indicate which section we are currently parsing
current_section = ""

# Iterate through the sentences to extract information
for sentence in doc.sents:
    # Detect section headers
    if "Summary" in sentence.text:
        current_section = "Summary"
    elif "Highlights" in sentence.text:
        current_section = "Highlights"
    elif "Education" in sentence.text:
        current_section = "Education"
    else:
        # Extract information based on the current section
        if current_section == "Summary":
            category = sentence.text.strip()
        elif current_section == "Highlights" and sentence.text.strip():
            skills.append(sentence.text.strip())
        elif current_section == "Education":
            education_info = sentence.text.strip()
            if education_info:
                # Split by ':' to extract degree and institution
                parts = education_info.split(':')
                if len(parts) == 2:
                    degree = parts[0].strip()
                    institution = parts[1].strip()
                    education = {"Degree": degree, "Institution": institution}

# Print the extracted information
print("Category (Job role):", category)
print("Skills:", skills)
print("Education:", education)

# Regular expression pattern to match college names and GPAs
pattern = r'([A-Z][\w\s]+(?: College)?)\s+(\d{4})\s+.*?GPA:\s*([\d.]+)'

# Find all matches in the text
matches = re.findall(pattern, text)

# Initialize variables to store extracted information
college_names = []
gpas = []

# Iterate through the matches and extract college names and GPAs
for match in matches:
    gpa = match[2]
    gpas.append(gpa)

# Regular expression pattern to match the college name before any number
pattern = r'Education\n(.*?)(?:\d+)'

# Find the match in the text
match = re.search(pattern, text, re.DOTALL)

# Extract the college name
if match:
    college_name = match.group(1).strip()
else:
    college_name = "Not found"

print("GPAs:", gpas)
print("College Name:", college_name)


Category (Job role): Financial Accountant specializing in financial planning, reporting and analysis within the Department of Defense.
Skills: ['Account reconciliations', 'Results-oriented Financial reporting Critical thinking', 'Accomplishments', 'Accounting operations professional\nAnalysis of financial systems', 'ERP (Enterprise Resource Planning) software.', 'Excellent facilitator\nServed on a tiger team which identified and resolved General Ledger postings in DEAMS totaling $360B in accounting adjustments.', 'This allowed for the first successful fiscal year-end close for 2012.', '...', '(rest of the text) ...']
Education: {'Degree': 'Associates', 'Institution': ''}
GPAs: ['3.41']
College Name: Northern Maine Community College


In [ ]:
import spacy
import re


# Load the English language model
nlp = spacy.load("en_core_web_sm")

# Input text
text = 'Associate\n \n: \nAccounting\n \nCity\n \n, \nState\n \n, \nUSA\nEmphasis in Business\n1994\n \nAssociates\n \n: \nAccounting\n \nCity\n \n, \nState\n \n, \nUSA\n \nGPA: \nGPA: 3.41\nAccounting GPA: 3.41 174 Hours, Quarter\nAttended Husson College, major Accounting 78 semester hours toward Bachelors degree.\nProfessional Military Comptroller School, 6wk, 4-98; Managerial Accounting I, 09-98; Interested-Based Bargaining Training for Management,\n24hrs, 09-01; Auditing Methods and Concepts 09-98; Organizational Leadership, 32hrs, 07-03; Management Development II, 32hrs, 07-03.\nCertifications\nCertified Defense Financial Manager, CDFM, May 2005\nInterests\nAmerican Society Of Military Comptrollers\nAdditional Information\nSkills\nAccounting; General Accounting; Accounts Payable; Program Management.'

# Process the text using spaCy
doc = nlp(text)

# Initialize variables to store extracted information
category = ""
skills = []
education = {}

# Flag to indicate which section we are currently parsing
current_section = ""

# Iterate through the sentences to extract information
for sentence in doc.sents:
    # Detect section headers
    if "Summary" in sentence.text:
        current_section = "Summary"
    elif "Highlights" in sentence.text:
        current_section = "Highlights"
    elif "Education" in sentence.text:
        current_section = "Education"
    else:
        # Extract information based on the current section
        if current_section == "Summary":
            category = sentence.text.strip()
        elif current_section == "Highlights" and sentence.text.strip():
            skills.append(sentence.text.strip())
        elif current_section == "Education":
            education_info = sentence.text.strip()
            if education_info:
                # Split by ':' to extract degree and institution
                parts = education_info.split(':')
                if len(parts) == 2:
                    degree = parts[0].strip()
                    institution = parts[1].strip()
                    education = {"Degree": degree, "Institution": institution}

# Print the extracted information
print("Category (Job role):", category)
print("Skills:", skills)
print("Education:", education)

# Regular expression pattern to match college names and GPAs
pattern = r'([A-Z][\w\s]+(?: College)?)\s+(\d{4})\s+.*?GPA:\s*([\d.]+)'

# Find all matches in the text
matches = re.findall(pattern, text)

# Initialize variables to store extracted information
college_names = []
gpas = []

# Iterate through the matches and extract college names and GPAs
for match in matches:
    gpa = match[2]
    gpas.append(gpa)

# Regular expression pattern to match the college name before any number
pattern = r'Education\n(.*?)(?:\d+)'

# Find the match in the text
match = re.search(pattern, text, re.DOTALL)

# Extract the college name
if match:
    college_name = match.group(1).strip()
else:
    college_name = "Not found"

print("GPAs:", gpas)
print("College Name:", college_name)


Category (Job role): 
Skills: []
Education: {}
GPAs: []
College Name: Not found


#### Proceeding ahead with pyresparser

## **Extract data from all pdfs and store in pandas dataframe**

In [ ]:
import os
import warnings
from pyresparser import ResumeParser
import pandas as pd

# Define the folder containing the PDFs
pdf_folder = '/content/data/data/ACCOUNTANT'

# Initialize an empty list to store extracted data
data_list = []

# Iterate through PDF files in the folder
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder, filename)
        # Suppress warnings
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            # Extract data from the PDF using ResumeParser
            pdf_data = ResumeParser(pdf_path).get_extracted_data()
        data_list.append(pdf_data)

# Convert the list of dictionaries into a pandas DataFrame
df1 = pd.DataFrame(data_list)
df1["name"] = "ACCOUNTANT"




In [ ]:
df1.head()

,name,email,mobile_number,skills,college_name,degree,designation,experience,company_names,no_of_pages,total_experience
0,ACCOUNTANT,None,None,"[Invoices, Banking, Testing, Adobe, Budgeting,...",None,None,None,"[Accountant 07/2012 to 08/2015 Company Name, I...",None,2,0.00
1,ACCOUNTANT,None,None,"[System, Administration, Training, Reporting, ...",None,None,None,"[01/2009 to Current, Accountant Company Name ï...",None,1,0.00
2,ACCOUNTANT,None,None,"[Invoices, Erp, Testing, Regulatory, Reports, ...",None,None,None,[Company Name July 2011 to November 2012 Accou...,None,5,19.33
3,ACCOUNTANT,None,None,"[Invoices, Reports, Updates, Technical, Specif...",None,None,None,[Company Name December 2009 to Current Staff A...,None,3,1.75
4,ACCOUNTANT,None,007-2008,"[Marketing, Budgeting, Reports, Business manag...",None,None,None,"[Accountant, July 2012 to October 2015 Company...",None,3,3.25


In [ ]:
main_df = df1

### Now have to do the same for all pdf's' in

In [ ]:
import os
import warnings
from pyresparser import ResumeParser
import pandas as pd

# Define the folder containing the PDFs
pdf_folder = '/content/data/data/ADVOCATE'

# Initialize an empty list to store extracted data
data_list = []

# Iterate through PDF files in the folder
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder, filename)
        # Suppress warnings
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            # Extract data from the PDF using ResumeParser
            pdf_data = ResumeParser(pdf_path).get_extracted_data()
        data_list.append(pdf_data)

# Convert the list of dictionaries into a pandas DataFrame
df1 = pd.DataFrame(data_list)
df1["name"] = "ADVOCATE"
main_df = pd.concat([main_df,df1])

In [ ]:
main_df.tail()

,name,email,mobile_number,skills,college_name,degree,designation,experience,company_names,no_of_pages,total_experience
113,DIGITAL MARKETING,None,None,"[Marketing, Healthcare, Mobile, Content, Inter...",None,None,None,"[Manager, Digital Marketing and Communications...",None,2,0.00
114,BUSINESS ADVOCATE,None,None,"[Invoices, Banking, Erp, Budgeting, Account ma...",None,None,None,None,None,4,0.00
115,Autism Spectrum,None,993-1995,"[Marketing, Reports, Math, Employee engagement...",None,None,None,[financial reports as well as conducting audit...,None,2,2.58
116,DRIVER Summary,None,None,"[International, Excel, Compliance, Retention, ...",None,None,None,"[DRIVER, April 2015 to March 2016 Company Name...",None,2,3.42
117,PRACTICUM EXPERIENCE,None,None,"[International, Healthcare, Research, Plan, Ad...",None,None,None,[],None,2,0.00


In [ ]:
# Define the folder containing the PDFs
pdf_folder = '/content/data/data/AGRICULTURE'

# Initialize an empty list to store extracted data
data_list = []

# Iterate through PDF files in the folder
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder, filename)
        # Suppress warnings
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            # Extract data from the PDF using ResumeParser
            pdf_data = ResumeParser(pdf_path).get_extracted_data()
        data_list.append(pdf_data)

# Convert the list of dictionaries into a pandas DataFrame
df1 = pd.DataFrame(data_list)

main_df = pd.concat([main_df,df1])

In [ ]:
# Define the folder containing the PDFs
pdf_folder = '/content/data/data/APPAREL'

# Initialize an empty list to store extracted data
data_list = []

# Iterate through PDF files in the folder
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder, filename)
        # Suppress warnings
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            # Extract data from the PDF using ResumeParser
            pdf_data = ResumeParser(pdf_path).get_extracted_data()
        data_list.append(pdf_data)

# Convert the list of dictionaries into a pandas DataFrame
df1 = pd.DataFrame(data_list)
main_df = pd.concat([main_df,df1])

In [ ]:
# Define the folder containing the PDFs
pdf_folder = '/content/data/data/AUTOMOBILE'

# Initialize an empty list to store extracted data
data_list = []

# Iterate through PDF files in the folder
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder, filename)
        # Suppress warnings
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            # Extract data from the PDF using ResumeParser
            pdf_data = ResumeParser(pdf_path).get_extracted_data()
        data_list.append(pdf_data)

# Convert the list of dictionaries into a pandas DataFrame
df1 = pd.DataFrame(data_list)
main_df = pd.concat([main_df,df1])

In [ ]:
# Define the folder containing the PDFs
pdf_folder = '/content/data/data/AVIATION'

# Initialize an empty list to store extracted data
data_list = []

# Iterate through PDF files in the folder
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder, filename)
        # Suppress warnings
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            # Extract data from the PDF using ResumeParser
            pdf_data = ResumeParser(pdf_path).get_extracted_data()
        data_list.append(pdf_data)

# Convert the list of dictionaries into a pandas DataFrame
df1 = pd.DataFrame(data_list)
main_df = pd.concat([main_df,df1])

In [ ]:
# Define the folder containing the PDFs
pdf_folder = '/content/data/data/BANKING'

# Initialize an empty list to store extracted data
data_list = []

# Iterate through PDF files in the folder
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder, filename)
        # Suppress warnings
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            # Extract data from the PDF using ResumeParser
            pdf_data = ResumeParser(pdf_path).get_extracted_data()
        data_list.append(pdf_data)

# Convert the list of dictionaries into a pandas DataFrame
df1 = pd.DataFrame(data_list)
main_df = pd.concat([main_df,df1])

In [ ]:
# Define the folder containing the PDFs
pdf_folder = '/content/data/data/BPO'

# Initialize an empty list to store extracted data
data_list = []

# Iterate through PDF files in the folder
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder, filename)
        # Suppress warnings
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            # Extract data from the PDF using ResumeParser
            pdf_data = ResumeParser(pdf_path).get_extracted_data()
        data_list.append(pdf_data)

# Convert the list of dictionaries into a pandas DataFrame
df1 = pd.DataFrame(data_list)
main_df = pd.concat([main_df,df1])

In [ ]:
# Define the folder containing the PDFs
pdf_folder = '/content/data/data/BUSINESS-DEVELOPMENT'

# Initialize an empty list to store extracted data
data_list = []

# Iterate through PDF files in the folder
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder, filename)
        # Suppress warnings
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            # Extract data from the PDF using ResumeParser
            pdf_data = ResumeParser(pdf_path).get_extracted_data()
        data_list.append(pdf_data)

# Convert the list of dictionaries into a pandas DataFrame
df1 = pd.DataFrame(data_list)
main_df = pd.concat([main_df,df1])

In [ ]:
# Define the folder containing the PDFs
pdf_folder = '/content/data/data/CHEF'

# Initialize an empty list to store extracted data
data_list = []

# Iterate through PDF files in the folder
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder, filename)
        # Suppress warnings
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            # Extract data from the PDF using ResumeParser
            pdf_data = ResumeParser(pdf_path).get_extracted_data()
        data_list.append(pdf_data)

# Convert the list of dictionaries into a pandas DataFrame
df1 = pd.DataFrame(data_list)
main_df = pd.concat([main_df,df1])

In [ ]:
# Define the folder containing the PDFs
pdf_folder = '/content/data/data/CONSTRUCTION'

# Initialize an empty list to store extracted data
data_list = []

# Iterate through PDF files in the folder
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder, filename)
        # Suppress warnings
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            # Extract data from the PDF using ResumeParser
            pdf_data = ResumeParser(pdf_path).get_extracted_data()
        data_list.append(pdf_data)

# Convert the list of dictionaries into a pandas DataFrame
df1 = pd.DataFrame(data_list)
main_df = pd.concat([main_df,df1])

In [ ]:
# Define the folder containing the PDFs
pdf_folder = '/content/data/data/CONSULTANT'

# Initialize an empty list to store extracted data
data_list = []

# Iterate through PDF files in the folder
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder, filename)
        # Suppress warnings
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            # Extract data from the PDF using ResumeParser
            pdf_data = ResumeParser(pdf_path).get_extracted_data()
        data_list.append(pdf_data)

# Convert the list of dictionaries into a pandas DataFrame
df1 = pd.DataFrame(data_list)
main_df = pd.concat([main_df,df1])

In [ ]:
# Define the folder containing the PDFs
pdf_folder = '/content/data/data/DESIGNER'

# Initialize an empty list to store extracted data
data_list = []

# Iterate through PDF files in the folder
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder, filename)
        # Suppress warnings
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            # Extract data from the PDF using ResumeParser
            pdf_data = ResumeParser(pdf_path).get_extracted_data()
        data_list.append(pdf_data)

# Convert the list of dictionaries into a pandas DataFrame
df1 = pd.DataFrame(data_list)
main_df = pd.concat([main_df,df1])

In [ ]:
# Define the folder containing the PDFs
pdf_folder = '/content/data/data/DIGITAL-MEDIA'

# Initialize an empty list to store extracted data
data_list = []

# Iterate through PDF files in the folder
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder, filename)
        # Suppress warnings
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            # Extract data from the PDF using ResumeParser
            pdf_data = ResumeParser(pdf_path).get_extracted_data()
        data_list.append(pdf_data)

# Convert the list of dictionaries into a pandas DataFrame
df1 = pd.DataFrame(data_list)
main_df = pd.concat([main_df,df1])

In [ ]:
# Define the folder containing the PDFs
pdf_folder = '/content/data/data/ENGINEERING'

# Initialize an empty list to store extracted data
data_list = []

# Iterate through PDF files in the folder
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder, filename)
        # Suppress warnings
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            # Extract data from the PDF using ResumeParser
            pdf_data = ResumeParser(pdf_path).get_extracted_data()
        data_list.append(pdf_data)

# Convert the list of dictionaries into a pandas DataFrame
df1 = pd.DataFrame(data_list)
main_df = pd.concat([main_df,df1])

In [ ]:
# Define the folder containing the PDFs
pdf_folder = '/content/data/data/FINANCE'

# Initialize an empty list to store extracted data
data_list = []

# Iterate through PDF files in the folder
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder, filename)
        # Suppress warnings
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            # Extract data from the PDF using ResumeParser
            pdf_data = ResumeParser(pdf_path).get_extracted_data()
        data_list.append(pdf_data)

# Convert the list of dictionaries into a pandas DataFrame
df1 = pd.DataFrame(data_list)
main_df = pd.concat([main_df,df1])

In [ ]:
# Define the folder containing the PDFs
pdf_folder = '/content/data/data/FITNESS'

# Initialize an empty list to store extracted data
data_list = []

# Iterate through PDF files in the folder
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder, filename)
        # Suppress warnings
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            # Extract data from the PDF using ResumeParser
            pdf_data = ResumeParser(pdf_path).get_extracted_data()
        data_list.append(pdf_data)

# Convert the list of dictionaries into a pandas DataFrame
df1 = pd.DataFrame(data_list)
main_df = pd.concat([main_df,df1])

In [ ]:
# Define the folder containing the PDFs
pdf_folder = '/content/data/data/HEALTHCARE'

# Initialize an empty list to store extracted data
data_list = []

# Iterate through PDF files in the folder
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder, filename)
        # Suppress warnings
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            # Extract data from the PDF using ResumeParser
            pdf_data = ResumeParser(pdf_path).get_extracted_data()
        data_list.append(pdf_data)

# Convert the list of dictionaries into a pandas DataFrame
df1 = pd.DataFrame(data_list)
main_df = pd.concat([main_df,df1])

In [ ]:
# Define the folder containing the PDFs
pdf_folder = '/content/data/data/HR'

# Initialize an empty list to store extracted data
data_list = []

# Iterate through PDF files in the folder
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder, filename)
        # Suppress warnings
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            # Extract data from the PDF using ResumeParser
            pdf_data = ResumeParser(pdf_path).get_extracted_data()
        data_list.append(pdf_data)

# Convert the list of dictionaries into a pandas DataFrame
df1 = pd.DataFrame(data_list)
main_df = pd.concat([main_df,df1])

In [ ]:
# Define the folder containing the PDFs
pdf_folder = '/content/data/data/INFORMATION-TECHNOLOGY'

# Initialize an empty list to store extracted data
data_list = []

# Iterate through PDF files in the folder
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder, filename)
        # Suppress warnings
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            # Extract data from the PDF using ResumeParser
            pdf_data = ResumeParser(pdf_path).get_extracted_data()
        data_list.append(pdf_data)

# Convert the list of dictionaries into a pandas DataFrame
df1 = pd.DataFrame(data_list)
main_df = pd.concat([main_df,df1])

In [ ]:
# Define the folder containing the PDFs
pdf_folder = '/content/data/data/PUBLIC-RELATIONS'

# Initialize an empty list to store extracted data
data_list = []

# Iterate through PDF files in the folder
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder, filename)
        # Suppress warnings
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            # Extract data from the PDF using ResumeParser
            pdf_data = ResumeParser(pdf_path).get_extracted_data()
        data_list.append(pdf_data)

# Convert the list of dictionaries into a pandas DataFrame
df1 = pd.DataFrame(data_list)
main_df = pd.concat([main_df,df1])

In [ ]:
# Define the folder containing the PDFs
pdf_folder = '/content/data/data/SALES'

# Initialize an empty list to store extracted data
data_list = []

# Iterate through PDF files in the folder
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder, filename)
        # Suppress warnings
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            # Extract data from the PDF using ResumeParser
            pdf_data = ResumeParser(pdf_path).get_extracted_data()
        data_list.append(pdf_data)

# Convert the list of dictionaries into a pandas DataFrame
df1 = pd.DataFrame(data_list)
main_df = pd.concat([main_df,df1])

In [ ]:
# Define the folder containing the PDFs
pdf_folder = '/content/data/data/TEACHER
# Initialize an empty list to store extracted data
data_list = []

# Iterate through PDF files in the folder
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder, filename)
        # Suppress warnings
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            # Extract data from the PDF using ResumeParser
            pdf_data = ResumeParser(pdf_path).get_extracted_data()
        data_list.append(pdf_data)

# Convert the list of dictionaries into a pandas DataFrame
df1 = pd.DataFrame(data_list)
main_df = pd.concat([main_df,df1])

In [ ]:
# Remove "Summary" from the "name" column
main_df['name'] = main_df['name'].str.replace(' Summary', '')
main_df = main_df.drop(['email','mobile_number', 'college_name','degree','designation','company_names',	'no_of_pages'], axis=1)

## PREPROCESSING TEXT DATA

In [ ]:
import pandas as pd
main_df = pd.read_csv("/content/PDF_DATA.csv")

In [ ]:
main_df

,name,skills,experience,total_experience
0,ACCOUNTANT,"['Invoices', 'Banking', 'Testing', 'Adobe', 'B...","['Accountant 07/2012 to 08/2015 Company Name',...",0.00
1,ACCOUNTANT,"['System', 'Administration', 'Training', 'Repo...","['01/2009 to Current', 'Accountant Company Nam...",0.00
2,ACCOUNTANT,"['Invoices', 'Erp', 'Testing', 'Regulatory', '...",['Company Name July 2011 to November 2012 Acco...,19.33
3,ACCOUNTANT,"['Invoices', 'Reports', 'Updates', 'Technical'...",['Company Name December 2009 to Current Staff ...,1.75
4,ACCOUNTANT,"['Marketing', 'Budgeting', 'Reports', 'Busines...","['Accountant', 'July 2012 to October 2015 Comp...",3.25
...,...,...,...,...
2376,TEACHER,"['Sales', 'Excel', 'Administration', 'Training...","['09/2016 to Current', 'Substitute Teacher Com...",0.00
2377,TEACHER,"['International', 'Testing', 'Policies', 'Cont...","['English Teacher Oct 2014 to Jun 2015', 'Comp...",0.67
2378,TEACHER,"['Psychology', 'Plan', 'Training', 'Cpr', 'Reg...",['to plan and schedule lessons that encourage ...,0.00
2379,TEACHER,"['Spanish', 'International', 'Interactive', 'P...",['Spanish teacher 10/2003 to Current Company N...,0.00


In [ ]:
from datasets import load_dataset
job_desc = load_dataset('jacob-hugging-face/job-descriptions',split='train')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
job_desc

Dataset({
    features: ['company_name', 'job_description', 'position_title', 'description_length', 'model_response'],
    num_rows: 853
})

In [ ]:
for job in job_desc:
  print(job)
  break

{'company_name': 'Google', 'job_description': 'minimum qualifications\nbachelors degree or equivalent practical experience years of experience in saas or productivity tools businessexperience managing enterprise accounts with sales cycles\npreferred qualifications\n years of experience building strategic business partnerships with enterprise customersability to work through and with a reseller ecosystem to scale the businessability to plan pitch and execute a territory business strategyability to build relationships and to deliver results in a crossfunctionalmatrixed environmentability to identify crosspromoting and uppromoting opportunities within the existing account baseexcellent account management writtenverbal communication strategic and analyticalthinking skills\nabout the job\nas a member of the google cloud team you inspire leading companies schools and government agencies to work smarter with google tools like google workspace search and chrome you advocate the innovative powe

In [ ]:
job_df =  pd.DataFrame.from_dict(job,orient="index").T


In [ ]:
job_df

,company_name,job_description,position_title,description_length,model_response
0,Google,minimum qualifications\nbachelors degree or eq...,Sales Specialist,2727,"{\n ""Core Responsibilities"": ""Responsible fo..."


In [ ]:
## Create dataframe of all job decriptions

In [ ]:
for job in job_desc:
  job_1 =  pd.DataFrame.from_dict(job,orient="index").T
  job_df= pd.concat([job_df,job_1])


In [ ]:
job_df.reset_index(drop=True, inplace=True)


In [ ]:
job_df

,company_name,job_description,position_title,description_length,model_response
0,Google,minimum qualifications\nbachelors degree or eq...,Sales Specialist,2727,"{\n ""Core Responsibilities"": ""Responsible fo..."
1,Google,minimum qualifications\nbachelors degree or eq...,Sales Specialist,2727,"{\n ""Core Responsibilities"": ""Responsible fo..."
2,Apple,description\nas an asc you will be highly infl...,Apple Solutions Consultant,828,"{\n ""Core Responsibilities"": ""as an asc you ..."
3,Netflix,its an amazing time to be joining netflix as w...,Licensing Coordinator - Consumer Products,3205,"{\n ""Core Responsibilities"": ""Help drive bus..."
4,Robert Half,description\n\nweb designers looking to expand...,Web Designer,2489,"{\n ""Core Responsibilities"": ""Designing webs..."
...,...,...,...,...,...
849,Menards,job description\n\nparttime\n\nmake big money ...,Management Internship,1122,"{\n ""Core Responsibilities"": ""Responsibiliti..."
850,Parker,responsibilities\nparkers internship program w...,Human Resources Internship - Corporate (Year-...,3840,"{\n ""Core Responsibilities"": ""Assist in gene..."
851,Borgen Project,the borgen project is an innovative national ...,Writer / Journalist Internship,897,"{\n ""Core Responsibilities"": ""Write one arti..."
852,Wyndham Destinations,put the world on vacation\n\nat wyndham destin...,Inbound Customer Service / Sales (Remote),4604,"{\n ""Core Responsibilities"": ""Answer inbound..."


In [ ]:
## Generate Embeddings

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def encode_text(text):
    input_ids = tokenizer.encode(text, add_special_tokens=True, max_length=512, truncation=True, padding='max_length', return_tensors='pt')
    with torch.no_grad():
        embeddings = model(input_ids)[0]
    return embeddings

resume_embeddings = [encode_text(text) for text in main_df["skills"]]
job_description_embeddings = [encode_text(text) for text in job_df["job_description"]]



In [ ]:
file_path = '/content/drive/My Drive/Capital/job_embeddings.pt'

torch.save(job_description_embeddings, file_path)


## Load the saved embeddings from the file
#loaded_embeddings = torch.load(file_path)

In [2]:
file_path = '/content/drive/My Drive/Capital/job_embeddings.pt'
job_embeddings = torch.load(file_path)
file_path = '/content/drive/My Drive/Capital/resume_embeddings.pt'
resume_embeddings = torch.load(file_path)


In [3]:
## Generate Cosine similiarity
import torch
from sklearn.metrics.pairwise import cosine_similarity
top_resumes_for_jobs = []

for job_embedding in job_embeddings:
    job_embedding = job_embedding.mean(dim=0)
    similarities = [cosine_similarity(job_embedding.reshape(1, -1), resume_embedding.mean(dim=0).reshape(1, -1)).item() for resume_embedding in resume_embeddings]

    similarity_pairs = list(enumerate(similarities))

    similarity_pairs.sort(key=lambda x: x[1], reverse=True)

    top_5_resumes = similarity_pairs[:5]

    top_resumes_for_jobs.append(top_5_resumes)
    break



In [4]:
top_resumes_for_jobs
## Top 5 resume mathces for first job

[[(1049, 0.40073782205581665),
  (1132, 0.388333797454834),
  (591, 0.38672101497650146),
  (1957, 0.38414567708969116),
  (1426, 0.3797473907470703)]]

#### Calculated top 5 resumes for a single job. To find out for all we have to run and loop. Not done due to computation time.

## SCRATCH CODES

In [ ]:
!pip install pdfminer.six
!pip install nltk

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 18.5 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [ ]:
from pdfminer.high_level import extract_text
import nltk
import re
import subprocess
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import json

In [ ]:
def extract_names(txt):
    person_names = []

    for sent in nltk.sent_tokenize(txt):
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
            if hasattr(chunk, 'label') and chunk.label() == 'PERSON':
                person_names.append(
                    ' '.join(chunk_leave[0] for chunk_leave in chunk.leaves())
                )

    return person_names

In [ ]:
names = extract_names(pdf_text)


In [ ]:
names

['Critical',
 'Accomplishments',
 'Name',
 'Resource Advisor',
 'Space Operations',
 'Finance System',
 'Name',
 'Air Force',
 'Defense Connect',
 'Name',
 'Reports Branch',
 'Finance System',
 'Limestone',
 'Limestone',
 'Staff',
 'Name',
 'Accounts Payable Branch City',
 'Workload',
 'Air Force',
 'Employee Performance',
 'Balance Scorecard',
 'Louis II',
 'Air Force',
 'Name',
 'Recon',
 'Vendor Pay Reports',
 'Recon Branch',
 'Name',
 'Accounts Payable Branch City',
 'Major Command',
 'Workload',
 'Louis II',
 'Air Force',
 'Vendor Pay Site Manager',
 'Resource Advisors',
 'Boot Camp',
 'Name',
 'Network Assistant Team City',
 'Defense Accounting',
 'Defense Accounting',
 'Name',
 'Accounts Control Branch City',
 'Finance System',
 'Major Command',
 'Data Automation',
 'Account Control',
 'Budget Ledger',
 'Allotment Ledger',
 'Finance Workload',
 'Finance Officer',
 'Education',
 'State',
 'State',
 'Quarter Attended Husson College',
 'Methods',
 'Certified Defense Financial Manag

In [ ]:
name_candidate=names[0] + ' ' + names[1].split(' ')[0]


In [ ]:
name_candidate

'Critical Accomplishments'

In [ ]:
!pip install spacy==2.3.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 25.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached thinc-7.4.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
  Using cached wasabi-0.10.1-py3-none-any.whl (26 kB)
  Using cached srsly-1.0.7-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (369 kB)
  Using cached catalogue-1.0.2-py2.py3-none-any.whl (16 kB)
  Using cached plac-1.1.3-py2.py3-none-any.whl (20 kB)
  error: subprocess-exited-with-error
  
  × Building wheel for spacy (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for spacy
Failed to build spacy
ERROR: Could not build wheels for spacy, which is required to install pyp

In [ ]:
pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 35.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 18.6 MB/s eta 0:00:00
  Using cached thinc-7.4.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
  Using cached wasabi-0.10.1-py3-none-any.whl (26 kB)
  Using cached srsly-1.0.7-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (369 kB)
  Using cached catalogue-1.0.2-py2.py3-none-any.whl (16 kB)
  Using cached plac-1.1.3-py2.py3-none-any.whl (20 kB)
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.1-py3-none-any.whl size=12047086 sha256=f0d152d998fce32ac4831a0342cf7aaaee6ebb6d5cf0e312f6f068681fd3d749
  Stored in directory: /root/.cache/pip/wheels/4f/1f/0e/16fae4b01d2d87454e0f484e58c48793efcf237f0894c1c4bd
Successfully built en-core-web-sm
  Attempting uninstall: wasabi
    Found existing installation: wasabi 1.1.2
    Uninstalling wasabi-1.1.2:
      Successfully uni

In [ ]:
import spacy

# Load the model
nlp = spacy.load("en_core_web_sm")

# Check if it loaded successfully
if "en_core_web_sm" in spacy.cli.info():
    print("Model 'en_core_web_sm' is correctly installed.")
else:
    print("Model 'en_core_web_sm' is not installed.")

Model 'en_core_web_sm' is not installed.


In [ ]:
nlp

In [ ]:
spacy

<module 'spacy' from '/usr/local/lib/python3.10/dist-packages/spacy/__init__.py'>

In [ ]:
!python -m spacy download en_core_web_sm


2023-09-14 18:24:12.464670: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 98.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
import os
import nltk

# Install SpaCy Dependencies
#os.system('python -m pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz')

# Install nltk Dependencies
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
# Install SpaCy Dependencies
os.system('python -m pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz')


0

In [ ]:
import pandas as pd
resume_data  = pd.read_csv("/content/Resume/Resume.csv")

In [ ]:
resume_data

,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR
...,...,...,...,...
2479,99416532,RANK: SGT/E-5 NON- COMMISSIONED OFFIC...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION
2480,24589765,"GOVERNMENT RELATIONS, COMMUNICATIONS ...","<div class=""fontsize fontface vmargins hmargin...",AVIATION
2481,31605080,GEEK SQUAD AGENT Professional...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION
2482,21190805,PROGRAM DIRECTOR / OFFICE MANAGER ...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION


In [ ]:
resume_data.Resume_str

0                HR ADMINISTRATOR/MARKETING ASSOCIATE\...
1                HR SPECIALIST, US HR OPERATIONS      ...
2                HR DIRECTOR       Summary      Over 2...
3                HR SPECIALIST       Summary    Dedica...
4                HR MANAGER         Skill Highlights  ...
                              ...                        
2479             RANK: SGT/E-5 NON- COMMISSIONED OFFIC...
2480             GOVERNMENT RELATIONS, COMMUNICATIONS ...
2481             GEEK SQUAD AGENT         Professional...
2482             PROGRAM DIRECTOR / OFFICE MANAGER    ...
2483             STOREKEEPER II       Professional Sum...
Name: Resume_str, Length: 2484, dtype: object